In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from matplotlib import pyplot as plt

from kmv import insolvency_probability

In [2]:
all_stock_data = yf.download(['WMT', 'CCC.WA'], start='2023-06-12', end='2024-06-13')
raw_close = pd.DataFrame({'ccc': all_stock_data.Close['CCC.WA'], 'wmt': all_stock_data.Close.WMT})
close = raw_close.interpolate("linear")
log_std = (np.log(close) - np.log(close.shift(1))).std()

[*********************100%%**********************]  2 of 2 completed


In [3]:
wmt = yf.Ticker("WMT")

wmt_params = {"L": wmt.info["totalDebt"],
              "r": (4.34 / 100 + 1) ** (1 / 250) - 1,
              "k": (wmt.info["dividendYield"] + 1) ** (1 / 250) - 1,
              "sigma_E": log_std.wmt,
              "E": wmt.info["currentPrice"] * wmt.info["sharesOutstanding"],
              "market_cap": wmt.info['marketCap']}

In [4]:
ccc = yf.Ticker("CCC.WA")

ccc_params = {"L": ccc.info["totalDebt"],
              "r": (5.75 / 100 + 1) ** (1 / 250) - 1,
              "k": 0,
              "sigma_E": log_std.ccc,
              "E": ccc.info["currentPrice"] * ccc.info["sharesOutstanding"],
              "market_cap": ccc.info['marketCap']}


In [5]:
insolvency_probability(20 * 250, wmt_params)

[9.94195119e-03 5.67524556e+11]


0.00018846297542329804

In [6]:
insolvency_probability(20 * 250, ccc_params)

[2.77022754e-02 9.31734383e+09]


0.4889506080113989

In [7]:
ccc_params['market_cap'] / 10 ** 9

8.436329984

In [8]:
%%capture

t = np.arange(0, 10, 0.05)
wmt_probs = [insolvency_probability(i * 250, wmt_params) for i in t]
plt.plot(t, wmt_probs)
plt.xlabel('t, [rok]')
plt.ylabel('prawdopodobieństwo niewypłacalności')
plt.grid()
plt.savefig("wmt.png")

In [10]:
%%capture

ccc_probs = [insolvency_probability(i * 250, ccc_params) for i in t]
plt.plot(t, ccc_probs)
plt.xlabel('t, [rok]')
plt.ylabel('prawdopodobieństwo niewypłacalności')
plt.grid()
plt.savefig("ccc.png")